In [ ]:
# link do colab: https://colab.research.google.com/drive/1gwOYmPriygEWK9kFpXLeKlvx3gqlSSNe#scrollTo=Ocurf4Uft6A_
# link do drive: https://drive.google.com/drive/folders/1pQvHvqWxBh3cbj2HtEeoMqLjEaSgSA09?usp=sharing
# !pip install opencv-python-headless
# !pip install albumentations
# !pip install mediapipe
# !pip install rembg
# !pip install onnxruntime

In [ ]:
import kagglehub
import os
import shutil
import cv2
import matplotlib.pyplot as plt
import albumentations as A
import numpy as np
import random
import mediapipe as mp
from rembg import remove
from google.colab import drive


# Leitura de Conjuntos de Dados

In [ ]:
## Dataframes disponibilizados no Notion
## Baixando datasets do Kaggle de ASL
path_ASL1 = kagglehub.dataset_download("grassknoted/asl-alphabet")
path_ASL2 = kagglehub.dataset_download("debashishsau/aslamerican-sign-language-aplhabet-dataset")

## Baixando datasets do Kaggle de Libras

path_libras1 = kagglehub.dataset_download("lexset/synthetic-asl-alphabet")
path_libras2 = kagglehub.dataset_download("williansoliveira/libras")

# Preparando paths para Data Augmentation

In [ ]:
## Entrando nas subpastas corretas para treino, de ASL1
path_ASL1 = os.path.join(path_ASL1, os.listdir(path_ASL1)[0])  # Pega a primeira subpasta encontrada
path_ASL1 = os.path.join(path_ASL1, os.listdir(path_ASL1)[0])  # Pega a primeira subpasta encontrada
print(path_ASL1)

In [ ]:
print(os.listdir(path_ASL1))

In [ ]:
## Entrando nas subpastas corretas para treino, de ASL2
path_ASL2 = os.path.join(path_ASL2, os.listdir(path_ASL2)[0])  # Pega a primeira subpasta encontrada
path_ASL2 = os.path.join(path_ASL2, os.listdir(path_ASL2)[0])
print(os.listdir(path_ASL2))

In [ ]:
path_libras1 = os.path.join(path_libras1, os.listdir(path_libras1)[0])
print(os.listdir(path_libras1))

In [ ]:
path_libras2 = os.path.join(path_libras2, os.listdir(path_libras2)[0])
print(os.listdir(path_libras2))

# Configuração de Mediapipe Hands para Extração de Landmarks

In [ ]:
## Medipipe Hands
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils ## Visualizando landmarks
mp_drawing_styles = mp.solutions.drawing_styles

# Função para adicionar Ruído Salt and Pepper


In [ ]:
'''
  Função para adicionar ruído salt ou pepper
  "ou", pois pode-se adicionar um por vez, e se quiser adicionar ambos, é só chamar a função 2 vezes
  *salt_or_pepper
      Se salt_or_pepper = 255 (pixel branco), adiciona ruído salt
      Se salt_or_pepper = 0 (pixel preto), adiciona ruído pepper
  *image
      imagem a ser aplicada o ruído

'''
def noise_salt_and_pepper(salt_or_pepper, image):
  ## valor de ruído de 0 a 0.5%
  prob = np.random.uniform(0.0, 0.005)
  output = np.copy(image)
  ## prob * image_size calcula a quantidade de pixels que serão alterados
  ## np.ceil arrendonda para cima (necessita-se que o valor de pixels seja inteiro)
  p_alterados = int(prob * image.size)
  ## gera-se coordenadas únicas
  coords_x = np.random.randint(0, image.shape[1], p_alterados)
  coords_y = np.random.randint(0, image.shape[0], p_alterados)

  ## aplica-se o ruído
  output[coords_y, coords_x] = salt_or_pepper

  return output

# Salvando Imagens no Drive

In [ ]:
## Criando Google Drive
drive.mount('/content/drive')

## Definindo caminho para salvar no Google Drive
save_path = "/content/drive/MyDrive/augmentadas/"
os.makedirs(save_path, exist_ok=True)


# Função para aplicar Data Augmentation

In [ ]:
'''
  Função para aplicar Data Augmentation
  *path
      Caminho da pasta que contém imagens
'''
def data_aug(path, save_path):
  for dir_ in os.listdir(path):
      ## Pegando-se caminho da pasta da letra. Por exemplo, letra "A"
      path_atual = os.path.join(path, dir_)
      os.makedirs(os.path.join(save_path, dir_), exist_ok=True)

      ## Onde será salvo o arquivo
      save_dir = os.path.join(save_path, dir_)

      ## Pega todas as imagens do diretório atual, ignorando-se arquivos ocultos
      img_files = [
          f for f in os.listdir(path_atual)
          if f.lower().endswith(('.png', '.jpg', '.jpeg')) and not f.startswith('.')
      ]

      if not img_files:
          print(f"Nenhuma imagem encontrada no diretório: {path_atual}")
          continue  ## Vai para próximo diretório se não houver imagens

      ## Seleciona imagem aleatória do conjunto de dados
      ## OBS: apenas foi selecionada uma aleatória, e não todas, para facilitar processamento.
      ## Quanto tivermos certeza da metodologia e dos sinais parecidos, aplicaremos a todas
      img_path = random.choice(img_files)
      ## Pega-se path completo da imagem
      img_full_path = os.path.join(path, dir_, img_path)

      ## Lendo a imagem e verificando se há erro
      img = cv2.imread(img_full_path)
      if img is None:
          print(f"Erro ao carregar imagem: {img_full_path}")
          continue  ## Vai para próximo diretório se não carregar imagens

      ## Converteno-se de BGR (padrão de leitura CV2) para RGB
      img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

      ## Aplicação das Transformações
      ## Utilização de Biblioteca "albumentations"
      transform = A.Compose([
        A.ColorJitter(brightness=0.4, contrast=0, saturation=0, hue=0, p=0.5),  ## Ajusta brilho
        A.ColorJitter(brightness=0, contrast=0.4, saturation=0, hue=0, p=0.5),  ## Ajusta contraste
        A.ColorJitter(brightness=0, contrast=0, saturation=0.4, hue=0, p=0.5),  ## Ajusta cor
        A.HorizontalFlip(p=0.5),  ## Flip horizontal
        A.Rotate(limit=10, p=0.5),  ## Rotação aleatória de ±10°
      ])

      ## Criando grid
      fig, axes = plt.subplots(2, 6, figsize=(18, 6))

      ## Exibindo imagem original
      axes[0, 0].imshow(img_rgb)
      axes[0, 0].axis("off")
      axes[0, 0].set_title("Original")

      ## Aplicando as transformações 10 vezes, para exemplificar
      for i in range(11):
          augmented = transform(image=img_rgb)["image"]

          ## Definindo opções de ruído Salt/Pepper
          noise_options = {
            "both": [255, 0],
            "salt": [255],
            "pepper": [0],
            "none": []
          }
          ## Esse código ou vai aplicar ambos os ruídos, nenhum ou 1 dos 2
          for value in noise_options[np.random.choice(list(noise_options))]:
            augmented = noise_salt_and_pepper(value, augmented)

            ## Converte imagem para BGR
            augmented_bgr = cv2.cvtColor(augmented, cv2.COLOR_RGB2BGR)

            ## Pega nome da imagem e extensão
            base_name, ext = os.path.splitext(img_path)
            aug_img_path = os.path.join(save_dir, f"{base_name}_aug_{i}{ext}")

            ## Salva imagem
            cv2.imwrite(aug_img_path, augmented_bgr)
          ## Organizando em Grid 2 por 6
          ax = axes[(i + 1) // 6, (i + 1) % 6]
          ax.imshow(augmented)
          ax.axis("off")
          ax.set_title(f"Variação {i+1}")



In [ ]:
data_aug(path_ASL2, save_path)

In [ ]:
'''
  Função para aplicar Apagamento de Background
  Semelhante à anterior, só que feita para apresentar imagens originais e sem fundo
  *path
      Caminho da pasta que contém imagens
'''
def mudanca_fundo_testes(path):
  for dir_ in os.listdir(path):
      path_atual = os.path.join(path, dir_)
      img_files = [
          f for f in os.listdir(path_atual)
          if f.lower().endswith(('.png', '.jpg', '.jpeg')) and not f.startswith('.')
      ]

      if not img_files:
          print(f"Nenhuma imagem encontrada no diretório: {path_atual}")
          continue

      img_path = random.choice(img_files)
      img_full_path = os.path.join(path, dir_, img_path)
      img = cv2.imread(img_full_path)

      if img is None:
          print(f"Erro ao carregar imagem: {img_full_path}")
          continue
      img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
      fig, axes = plt.subplots(1, 2, figsize=(18, 6))

      img_sem_bg = remove(img)
      ## Imagem Original
      axes[0].imshow(img_rgb)
      axes[0].axis("off")
      axes[0].set_title("Original")

      ## Imagem sem Fundo
      axes[1].imshow(img_sem_bg)
      axes[1].axis("off")
      axes[1].set_title("Sem Fundo")



In [ ]:
mudanca_fundo_testes(path_ASL2)